In [1]:
import plotly.express as px
import pandas as pd
import polars as pl
import numpy as np
from math import ceil, log2

In [112]:
df:pl.DataFrame = pl.read_csv("/home/michael/Datasets/temp/script_job_df50654add33025f3714f49a6e3dd020_0.csv")
df.describe()

statistic,Cnt,Stake
str,f64,f64
"""count""",13346.0,13346.0
"""null_count""",0.0,0.0
"""mean""",307.875993,48237.086765
"""std""",6133.584392,303626.368438
"""min""",1.0,0.4835
"""25%""",1.0,3285.0
"""50%""",2.0,7112.0
"""75%""",11.0,15426.0
"""max""",370419.0,1.352925e7


In [110]:

total = int(df['Cnt'].sum())

df =  (
        df
        .filter([pl.col('Stake')>0]) #
        .with_columns([
                pl.col('Stake').cast(pl.Int32).cast(pl.String).cast(pl.Categorical).alias('Stk'),
                (pl.col('Cnt')/total).round(6).alias('Per')
        ])
)

v1 = df.to_pandas()
v1

,Cnt,Stake,Stk,Per
0,2,4.835000e-01,0,0.000000
1,5327,1.000000e+00,1,0.001296
2,5416,2.000000e+00,2,0.001318
3,81342,3.000000e+00,3,0.019796
4,27502,4.000000e+00,4,0.006693
...,...,...,...,...
13341,1,7.628000e+06,7628000,0.000000
13342,1,8.000000e+06,8000000,0.000000
13343,2,8.823529e+06,8823529,0.000000
13344,1,9.000000e+06,9000000,0.000000


In [111]:
dfX = df.filter([pl.col('Per')>0.0005]).select(pl.all())
left = int(dfX['Cnt'].sum())
print(f"Data coverage==>{np.round(left/total,2)},{len(dfX)}")

Data coverage==>0.86,191


In [114]:
dfX.describe()

statistic,Cnt,Stake,Stk,Per
str,f64,f64,str,f64
"""count""",191.0,191.0,"""191""",191.0
"""null_count""",0.0,0.0,"""0""",0.0
"""mean""",18429.701571,724.104712,null,0.004485
"""std""",48012.97829,2481.085318,null,0.011685
"""min""",2071.0,1.0,null,0.000504
"""25%""",2755.0,49.0,null,0.00067
"""50%""",4793.0,96.0,null,0.001166
"""75%""",8742.0,250.0,null,0.002128
"""max""",370419.0,20000.0,null,0.09015


In [115]:
fig = px.bar(dfX.to_pandas(), x='Stk',y='Cnt')
fig.show()

## Approach 2

In [119]:
fig = px.histogram(df.to_pandas(), x='Stake',nbins=40)
fig.show()

# Approach 3


### Freedman-Diaconis Rule

k is calculated as:
$$
k = \frac{2*IRQ(x)}{n^\frac{1}{3}}
$$

In [120]:
def calculate_bins(data, column):
    """
    Calculate the number of bins using the Freedman-Diaconis rule.

    Parameters:
    - data: pandas.DataFrame
    - column: str, the column name for which to calculate the number of bins

    Returns:
    - int, the suggested number of bins
    """
    # Extract the relevant data
    col_data = data[column].dropna()  # Remove NaN values for accurate calculations

    # Calculate the interquartile range (IQR)
    Q1 = np.percentile(col_data, 25)
    Q3 = np.percentile(col_data, 75)
    IQR = Q3 - Q1

    # Calculate bin width using the Freedman-Diaconis rule
    # Avoid division by zero by handling the zero-IQR case
    if IQR == 0:
        bin_width = np.ptp(col_data) / len(col_data)  # Peak-to-peak range divided by the number of observations
    else:
        bin_width = 2 * IQR / (len(col_data) ** (1/3))

    # Calculate the number of bins
    n_bins = int(np.ceil((col_data.max() - col_data.min()) / bin_width))
    
    return n_bins

In [122]:
n_bins_y = calculate_bins(df.to_pandas(), 'Stake')
n_bins_y

13218

In [126]:

n_bins_y = calculate_bins(df.to_pandas(), 'Stake')
fig = px.histogram(df.to_pandas(), x='Stake',nbins=n_bins_y)
fig.show()


Sturges' Formula: This formula is a commonly used rule of thumb that calculates the number of bins as:

$$
k =  \text{log}_2(n)+1
$$


In [123]:
def calculate_bins_sturges(data, column):
    """
    Calculate the number of bins using Sturges' formula.

    Parameters:
    - data: pandas.DataFrame
    - column: str, the column name for which to calculate the number of bins

    Returns:
    - int, the suggested number of bins
    """
    # Number of observations
    n = len(data[column].dropna())  # Drop NaN to avoid counting them

    # Apply Sturges' formula
    k = ceil(log2(n) + 1)

    return k

In [124]:
n_bins_y = calculate_bins_sturges(df.to_pandas(), 'Stake')
n_bins_y

15

In [125]:
fig = px.histogram(df.to_pandas(), x='Stake',nbins=n_bins_y)
fig.show()